In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import os
import sys
import chaospy

In [ ]:
sys.path.insert(0, os.getcwd() + "/../scripts")
import _helpers as h

plt.style.use(["../matplotlibrc","bmh"])
%matplotlib inline

In [ ]:
from sobol import calculate_sobol, calculate_sobol_m2

In [ ]:
with open("../config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
with open("../config.pypsaeur.yaml", 'r') as stream:
    peur_config = yaml.safe_load(stream)
    TECH_COLORS = peur_config["plotting"]["tech_colors"]
    NICE_NAMES = peur_config["plotting"]["nice_names"]

In [ ]:
uncertainties = config["uncertainties"]
distribution = h.NamedJ(uncertainties)

In [ ]:
epsilons = [0.0, 0.01, 0.02, 0.04, 0.06, 0.08]
sobol = {}
for dimension in ["wind", "solar", "battery", "H2", "transmission"]:
    for epsilon in epsilons:
        print(epsilon, end=' ')
        if epsilon == 0.0:
            surrogate = h.NamedPoly.from_txt(f"../results/pce/polynomial-low-min-cost-0.0.txt")
        else:
            surrogate = h.NamedPoly.from_txt(f"../results/pce/polynomial-low-min-{dimension}-{epsilon}.txt")

        variance = chaospy.Var(surrogate.poly, distribution.J)
        sobol[epsilon] = calculate_sobol(surrogate, distribution, sobol='m') * variance

    df = pd.concat(sobol, axis=1)
    
    for attr in ["wind", "solar", "battery", "H2", "transmission"]:

        dff = df.xs(attr, level=1, axis=1).T

        c = dff.columns.map(TECH_COLORS)
        dff.columns = dff.columns.map(NICE_NAMES)
        dff.index = [i*100 for i in dff.index]

        fig, ax = plt.subplots(figsize=(4.5,3))
        dff.plot(ax=ax, kind="area", linewidth=0, color=c)
        plt.grid(False)
        plt.box(False)
        plt.legend(frameon=False, bbox_to_anchor=(1,1))
        plt.xlabel(r"$\varepsilon$ [%]")
        plt.ylabel(f"Variance of {NICE_NAMES[attr]}")
        
        plt.title(f"Min {NICE_NAMES[dimension]}")
        
        plt.savefig(f"graphics/sobolepsilon/se-{dimension}-{attr}.pdf", bbox_inches='tight')
        
        plt.close(fig)